In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import osmnx as ox
import geopandas as gpd
from geopy.distance import geodesic
import time
from shapely.geometry import Point
from shapely.ops import unary_union
from geopy.distance import geodesic

In [3]:
archivos = {
    'Torrevieja': 'Spain-torrevieja_Property_Extended_Match_2023-11-23',
    'San Miguel de Salinas': 'Spain-san-miguel-de-salinas_Property_Extended_Match_2023-11-23',
    'Montesinos': 'Spain-los-montesinos_Property_Extended_Match_2023-11-23',
    'Orihuela': 'Spain-orihuela_Property_Extended_Match_2023-11-23',
    'Pilar de la Horadada': 'Spain-pilar-de-la-horadada_Property_Extended_Match_2023-11-23',
    'Rojales': 'Spain-rojales_Property_Extended_Match_2023-11-23',
    'Guardamar': 'Spain-guardamar-del-segura_Property_Extended_Match_2023-11-23',

}

In [4]:
df_combined = pd.DataFrame()
for carpeta, archivo in archivos.items():
    # Leer el archivo CSV
    df = pd.read_csv(f'{carpeta}/{archivo}/{archivo}.csv')
    
    # Crear un nuevo DataFrame con los datos de ambos tipos de propiedad
    df_combined = pd.concat([df_combined, df], ignore_index=True)

C:\Users\danim\AppData\Local\Temp\ipykernel_10288\2340592474.py:4: DtypeWarning: Columns (48,56,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{carpeta}/{archivo}/{archivo}.csv')
C:\Users\danim\AppData\Local\Temp\ipykernel_10288\2340592474.py:4: DtypeWarning: Columns (4,12,25,26,38,39,48,59,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{carpeta}/{archivo}/{archivo}.csv')


In [5]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31965 entries, 0 to 31964
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Property ID                    31965 non-null  object 
 1   Listing Title                  31959 non-null  object 
 2   Property Type                  31955 non-null  object 
 3   Listing Type                   31961 non-null  object 
 4   Created Date                   25844 non-null  object 
 5   Last Scraped Date              31965 non-null  object 
 6   Country                        31964 non-null  object 
 7   State                          29004 non-null  object 
 8   City                           31965 non-null  object 
 9   Currency Native                25834 non-null  object 
 10  Average Daily Rate (USD)       11266 non-null  float64
 11  Average Daily Rate (Native)    10023 non-null  float64
 12  Annual Revenue LTM (USD)       16367 non-null 

In [6]:
pois = gpd.read_file("pois_vega_baja.geojson")

In [7]:
# Filtrar los POIs para incluir solo geometrías de tipo Point
# pois_points = pois[pois['geometry'].geom_type == 'Point']

# Crear una función para calcular la distancia y el ID del POI más cercano
def calcular_distancia_y_id_mas_cercano(lat, lon, pois):
    distancias = pois['geometry'].apply(lambda geom: geodesic((lat, lon), (geom.y, geom.x)).meters if geom.geom_type == 'Point' else geodesic((lat, lon), (geom.centroid.y, geom.centroid.x)).meters)
    idx_min = distancias.idxmin()
    return distancias.min(), pois.loc[idx_min, 'id']

# Crear una función para calcular la distancia y el ID del POI más cercano por tipo
def calcular_distancia_y_id_por_tipo(lat, lon, pois):
    distancias_por_tipo = {}
    ids_por_tipo = {}
    for tipo in pois['amenity'].unique():
        pois_tipo = pois[pois['amenity'] == tipo]
        distancias = pois_tipo['geometry'].apply(lambda geom: geodesic((lat, lon), (geom.y, geom.x)).meters if geom.geom_type == 'Point' else geodesic((lat, lon), (geom.centroid.y, geom.centroid.x)).meters)
        idx_min = distancias.idxmin()
        distancias_por_tipo[tipo] = distancias.min()
        ids_por_tipo[tipo] = pois_tipo.loc[idx_min, 'id']
    return distancias_por_tipo, ids_por_tipo



# Calcular la distancia y el ID del POI más cercano por tipo para cada piso
# Ensure all geometries are points by using centroids for non-point geometries
# pois['geometry'] = pois['geometry'].apply(lambda geom: geom if geom.geom_type == 'Point' else geom.centroid)

# Calcular la distancia y el ID del POI más cercano para cada piso
df_combined[['Distancia_POI_Mas_Cercano', 'ID_POI_Mas_Cercano']] = df_combined.apply(
    lambda row: pd.Series(calcular_distancia_y_id_mas_cercano(row['Latitude'], row['Longitude'], pois)), axis=1
)

# Calculate the distance and ID of the nearest POI by type for each property
df_combined[['Distancia_POI_Por_Tipo', 'ID_POI_Por_Tipo']] = df_combined.apply(
    lambda row: pd.Series(calcular_distancia_y_id_por_tipo(row['Latitude'], row['Longitude'], pois)), axis=1
)

In [11]:
df_combined.to_json("df_combined.json")

In [12]:
amenities = pois['amenity'].unique()

In [24]:
amenities

array(['parking', 'restaurant', 'lugares_de_culto', 'bar_cafe_pub',
       'bank', 'ocio', 'fast_food', 'taxi', 'gambling', 'police',
       'library', 'car_rental', 'atm', 'bicycle_rental', 'townhall',
       'fountain', 'ice_cream', 'centros_sociales', 'marketplace',
       'vida_nocturna', 'bus_station', 'boat_rental'], dtype=object)

In [21]:
for amenity in amenities:
    distancia = 'distancia_'+amenity
    id_amenity = 'id_'+amenity
    df_combined[distancia] = df_combined['Distancia_POI_Por_Tipo'].apply(lambda x: x.get(amenity, None))
    df_combined[id_amenity] = df_combined['ID_POI_Por_Tipo'].apply(lambda x: x.get(amenity, None))

In [28]:
for column in df_combined.columns: print(column)

Property ID
Listing Title
Property Type
Listing Type
Created Date
Last Scraped Date
Country
State
City
Currency Native
Average Daily Rate (USD)
Average Daily Rate (Native)
Annual Revenue LTM (USD)
Annual Revenue LTM (Native)
Occupancy Rate LTM
Number of Bookings LTM
Number of Reviews
Bedrooms
Bathrooms
Max Guests
Calendar Last Updated
Response Rate
Airbnb Response Time (Text)
Airbnb Superhost
HomeAway Premier Partner
Cancellation Policy
Security Deposit (USD)
Security Deposit (Native)
Cleaning Fee (USD)
Cleaning Fee (Native)
Extra People Fee (USD)
Extra People Fee (Native)
Published Nightly Rate (USD)
Published Monthly Rate (USD)
Published Weekly Rate (USD)
Check-in Time
Checkout Time
Minimum Stay
Count Reservation Days LTM
Count Available Days LTM
Count Blocked Days LTM
Number of Photos
Instantbook Enabled
Listing URL
Listing Main Image URL
Listing Images
Latitude
Longitude
Exact Location
Overall Rating
Airbnb Communication Rating
Airbnb Accuracy Rating
Airbnb Cleanliness Rating
Airbn

In [29]:
df_combined.drop(columns=['Distancia_POI_Por_Tipo', 'ID_POI_Por_Tipo'], inplace=True)
df_combined.to_csv("df_combined.csv", index=False)